# HW5 LSTM and GRU

## Settings

In [3]:
import torch
from torchtext import data
from torchtext import datasets
import random
import time

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(tensor_type=torch.FloatTensor)

train, test = datasets.IMDB.splits(TEXT, LABEL)

train, valid = train.split(random_state=random.seed(SEED))

downloading aclImdb_v1.tar.gz


In [4]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [01:41, 8.51MB/s]                               
100%|██████████| 400000/400000 [00:16<00:00, 24902.09it/s]


In [5]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

## LSTM

In [6]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid. dim]
        #cell = [num layers * num directions, batch size, hid. dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
                
        #hidden [batch size, hid. dim * num directions]
            
        return self.fc(hidden.squeeze(0))

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [8]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [9]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4096, -0.5753,  0.1126,  ...,  0.4092,  0.1856,  0.1066],
        [ 0.2110, -0.2472,  0.6508,  ..., -0.1627,  0.4507, -1.1627],
        [-0.2379, -0.1095,  0.4314,  ...,  0.6665,  0.3200,  0.8872]])

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [11]:
criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = criterion.to(device)

In [12]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [13]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
N_EPOCHS = 5
start = time.time()
for epoch in range(N_EPOCHS):

    lstm_train_loss, lstm_train_acc = train(model, train_iterator, optimizer, criterion)
    lstm_valid_loss, lstm_valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss - LSTM: {lstm_train_loss:.3f}, Train Acc - LSTM: {lstm_train_acc*100:.2f}%, LSTM Val. Loss: {lstm_valid_loss:.3f}, LSTM Val. Acc: {lstm_valid_acc*100:.2f}%')
end = time.time()
print ('time elapsed:',end - start)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ec2-user/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Epoch: 01, Train Loss - LSTM: 0.676, Train Acc - LSTM: 58.68%, LSTM Val. Loss: 0.669, LSTM Val. Acc: 62.38%
Epoch: 02, Train Loss - LSTM: 0.649, Train Acc - LSTM: 61.41%, LSTM Val. Loss: 0.697, LSTM Val. Acc: 50.85%
Epoch: 03, Train Loss - LSTM: 0.647, Train Acc - LSTM: 64.18%, LSTM Val. Loss: 0.622, LSTM Val. Acc: 69.58%
Epoch: 04, Train Loss - LSTM: 0.600, Train Acc - LSTM: 69.10%, LSTM Val. Loss: 0.528, LSTM Val. Acc: 74.27%
Epoch: 05, Train Loss - LSTM: 0.507, Train Acc - LSTM: 76.21%, LSTM Val. Loss: 0.495, LSTM Val. Acc: 78.82%
time elapsed: 1554.9054231643677


In [16]:
lstm_test_loss, lstm_test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss - LSTM: {lstm_test_loss:.3f}, Test Acc - LSTM: {lstm_test_acc*100:.2f}%')

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss - LSTM: 0.508, Test Acc - LSTM: 77.94%


In [17]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = F.sigmoid(model(tensor))
    return prediction.item()

In [18]:
predict_sentiment("This film is terrible")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.2622326612472534

In [19]:
predict_sentiment("This film is great")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.8033835291862488

## GRU

In [20]:
import torch.nn as nn

class RNN_GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid. dim]
        #cell = [num layers * num directions, batch size, hid. dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
                
        #hidden [batch size, hid. dim * num directions]
            
        return self.fc(hidden.squeeze(0))

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model_gru = RNN_GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

pretrained_embeddings = TEXT.vocab.vectors
model_gru.embedding.weight.data.copy_(pretrained_embeddings)

optimizer = optim.Adam(model_gru.parameters())

criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_gru = model_gru.to(device)
criterion = criterion.to(device)

In [21]:
N_EPOCHS = 5
start = time.time()
for epoch in range(N_EPOCHS):

    gru_train_loss, gru_train_acc = train(model_gru, train_iterator, optimizer, criterion)
    gru_valid_loss, gru_valid_acc = evaluate(model_gru, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss - GRU: {gru_train_loss:.3f}, Train Acc - GRU: {gru_train_acc*100:.2f}%, GRU Val. Loss: {gru_valid_loss:.3f}, GRU Val. Acc: {gru_valid_acc*100:.2f}%')
end = time.time()
print ('time elapsed:',end - start)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ec2-user/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Epoch: 01, Train Loss - GRU: 0.652, Train Acc - GRU: 59.57%, GRU Val. Loss: 0.420, GRU Val. Acc: 82.03%
Epoch: 02, Train Loss - GRU: 0.341, Train Acc - GRU: 85.53%, GRU Val. Loss: 0.267, GRU Val. Acc: 89.29%
Epoch: 03, Train Loss - GRU: 0.221, Train Acc - GRU: 91.39%, GRU Val. Loss: 0.245, GRU Val. Acc: 90.38%
Epoch: 04, Train Loss - GRU: 0.152, Train Acc - GRU: 94.46%, GRU Val. Loss: 0.270, GRU Val. Acc: 89.47%
Epoch: 05, Train Loss - GRU: 0.117, Train Acc - GRU: 95.73%, GRU Val. Loss: 0.288, GRU Val. Acc: 90.36%
time elapsed: 1359.2399842739105


In [26]:
print ('time elapsed:',end - start)

time elapsed: 1359.2399842739105


In [27]:
gru_test_loss, gru_test_acc = evaluate(model_gru, test_iterator, criterion)

print(f'GRU Test Loss: {gru_test_loss:.3f}, GRU Test Acc: {gru_test_acc*100:.2f}%')

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


GRU Test Loss: 0.355, GRU Test Acc: 87.32%


In [34]:
import spacy
nlp = spacy.load('en')

def gru_predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = F.sigmoid(model_gru(tensor))
    return prediction.item()

In [35]:
gru_predict_sentiment("This film is terrible")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.1603315770626068

In [36]:
gru_predict_sentiment("This film is great")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.9836633205413818

## Conclusion

According to the result, GRU generally gives larger validation and test accuracy than LSTM and uses less time to complete training, which makes GRU a better model for use in this case. A simple example of prediction also shows that GRU seems to distinguish positive and negative reviews more clearly.